In [1]:
from analysis import tide_constituents
import xarray as xr
import numpy as np
import utils
from datetime import datetime

In [2]:
res = xr.open_dataset('/home/taher/Documents/C2_A1_S1_R25_D0_TPG/FlowFM_map.nc')

In [3]:
vel = res.mesh2d_ucy

In [4]:
wl = res.mesh2d_s1

In [5]:
x_center = 2.5e4
y_mouth = 1.25e5

In [138]:
a=[1,2]
b=[3,4]
c=[]
c.append([a, b])
c.append([a, b])

d, e = np.column_stack(c[0])
d, e

(array([1, 3]), array([2, 4]))

In [6]:
nx, ny = res.mesh2d_face_x.values, res.mesh2d_face_y.values
idx_xm = np.where((nx > x_center - 300) & (nx < x_center))[0]
idx_ym = np.where((ny > y_mouth - 500) & (ny < y_mouth))[0]
mouth_idx = np.intersect1d(idx_xm, idx_ym)

In [7]:
y_center = ny[idx_xm[::4]]
idx_sort = np.argsort(y_center)[::-1]
y_center = y_center[idx_sort]

In [8]:
elevation = np.array([w[mouth_idx].values[0] for w in wl])
velocity = np.array([u[mouth_idx].values[0] for u in vel])

In [143]:
elevation.shape

(506,)

In [9]:
dates = wl.time.values.astype(int) * 1e-9

In [10]:
dates = np.array([datetime.utcfromtimestamp(t) for t in dates])

In [14]:
elevations = np.column_stack([wl.isel(time=t)[idx_xm[::4]].values for t in range(wl.time.shape[0])])

In [140]:
elevations.shape

(280, 506)

In [15]:
wl_constituents = [tide_constituents(dates, el) for el in elevations]

In [16]:
amps = np.column_stack([(con.r['M2'], con.r['M4']) for con in wl_constituents])
phases = np.column_stack([(con.phase['M2'], con.phase['M4']) for con in wl_constituents])

In [139]:
np.column_stack([(1,2), (3,4)])

array([[1, 3],
       [2, 4]])

In [11]:
import matplotlib.dates as mdates

fig, gs, canvas = utils.make_canvas(8, 5)
ax = fig.add_subplot(gs[0])

ax.plot(dates, velocity)
fig.autofmt_xdate()

ax.set_title('Velocity at the mouth of the river')
#ax.set_ylim(ymin, ymax)
ax.set_xlabel('')
ax.set_ylabel('Velocity (m/s)')

ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
#ax.legend(loc='upper right')

canvas.print_figure('vel.png', format="png", dpi=300)

/home/taher/applications/anaconda/envs/claw/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [177]:
arr=['TTT', 'TFT', 'FFT', 'TTF']
[arr.index(a) for a in arr if a[2] == 'T']

[0, 1, 2]

In [12]:
import matplotlib.dates as mdates

fig, gs, canvas = utils.make_canvas(8, 5)
ax = fig.add_subplot(gs[0])

ax.plot(dates, elevation)
fig.autofmt_xdate()

ax.set_title('Water level at the mouth of the river')
#ax.set_ylim(ymin, ymax)
ax.set_xlabel('')
ax.set_ylabel('Water Level (m)')

ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
#ax.legend(loc='upper right')

canvas.print_figure('wl.png', format="png", dpi=300)

In [116]:
def make_canvas(width, height, nx=1, ny=1):
    from matplotlib.figure import Figure
    from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
    import matplotlib.gridspec as gridspec

    # latexify(width, height)
    fig = Figure(figsize=(width, height), frameon=True)
    canvas = FigureCanvas(fig)
    gs = gridspec.GridSpec(nx,
                           ny,
                           left=0.15,
                           right=0.95,
                           bottom=0.15,
                           top=0.95,
                           wspace=0.0,
                           hspace=0.0,
                           width_ratios=None,
                           height_ratios=None)
    return fig, gs, canvas

In [148]:
idx_sort.max()

279

In [123]:
import matplotlib.dates as mdates

fig, gs, canvas = make_canvas(10, 9, nx=2, ny=1)

ax = [fig.add_subplot(g) for g in gs]

ax[0].plot(y_center, amps[0][idx_sort])
#fig.autofmt_xdate()

#ax[0].set_title('The $M_2$ tidal constituents of Water level along the center of the domain')
#ax.set_xlim(0, 0.2e6)

#ax.legend(loc='upper right')
ax[1].plot(y_center, phases[0][idx_sort])
#fig.autofmt_xdate()
fig.suptitle('The $M_2$ tidal constituents of water level along the center of the domain',
             y=0.98, horizontalalignment='center', verticalalignment='top', fontsize = 12)
#ax[1].set_title('The $M_2$ tidal constituents of Water level along the center of the domain')
#ax.set_xlim(0, 0.2e6)
#ax[0].set_xlabel('Distance (m)')
ax[0].set_xticks([])
ax[0].set_ylabel('$M_2$ ampl. (m/s)')

ax[0].ticklabel_format(style='sci', axis='x', scilimits=(0, 3))
ax[1].set_xlabel('Distance (m)')
ax[1].set_ylabel('$M_2$ phase (deg)')

ax[1].ticklabel_format(style='sci', axis='x', scilimits=(0, 3))
gs.tight_layout(fig)

canvas.print_figure('m2amp.png', format="png", dpi=300)

In [52]:
import matplotlib.dates as mdates

fig, gs, canvas = make_canvas(8, 5)
ax = fig.add_subplot(gs[0])

ax.plot(y_center, amps[1][idx_sort])
#fig.autofmt_xdate()

ax.set_title('The $M_4$ tidal constituents of Water level along the center of the domain')
#ax.set_xlim(0, 0.2e6)
ax.set_xlabel('Distance (m)')
ax.set_ylabel('$M_4$ ampl. (m/s)')

ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 3))
#ax.legend(loc='upper right')

canvas.print_figure('m4amp.png', format="png", dpi=300)

In [20]:
import matplotlib.dates as mdates

fig, gs, canvas = utils.make_canvas(8, 5)
ax = fig.add_subplot(gs[0])

ax.plot(y_center, phases[0][idx_sort])
#fig.autofmt_xdate()

ax.set_title('The $M_2$ tidal constituents of Water level along the center of the domain')
#ax.set_xlim(0, 0.2e6)
ax.set_xlabel('Distance (m)')
ax.set_ylabel('$M_2$ phase (rad)')

ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 3))
#ax.legend(loc='upper right')

canvas.print_figure('m2phase.png', format="png", dpi=300)

In [21]:
import matplotlib.dates as mdates

fig, gs, canvas = utils.make_canvas(8, 5)
ax = fig.add_subplot(gs[0])

ax.plot(y_center, phases[1][idx_sort])
#fig.autofmt_xdate()

ax.set_title('The $M_4$ tidal constituents of Water level along the center of the domain')
#ax.set_xlim(0, 0.2e6)
ax.set_xlabel('Distance (m)')
ax.set_ylabel('$M_4$ phase (rad)')

ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 3))
#ax.legend(loc='upper right')

canvas.print_figure('m4phase.png', format="png", dpi=300)